In [8]:
import pymongo, spacy, gensim
from datetime import datetime
import numpy as np
from spacy.tokens.doc import Doc

In [2]:
db = pymongo.MongoClient()
dail = db.texts.dail
houses = db.houses.houses

In [6]:
%time nlp = spacy.load("en")

CPU times: user 7.11 s, sys: 1.98 s, total: 9.09 s
Wall time: 9.12 s


Retrieve the start and end dates for an individual Dáil, and retrieve documents for that dail.

In [3]:
dail4 = houses.find_one({"house_type":'0', 'house_num':"4"}, { "house_num":1, "start_term":1, "end_term":1})
print("Dáil:", dail4['house_num'], "\nStart date:", dail4['start_term'].date(), "\nEnd date:", dail4['end_term'].date())

Dáil: 4 
Start date: 1923-09-19 
End date: 1927-05-23


In [4]:
agg = dail.aggregate([{"$match": 
                     {"date": {"$gte": dail4["start_term"], 
                               "$lte": dail4['end_term']}
                     }}, 
                    {"$group": {"_id": None, 
                                "sum": {"$sum": "$len_doc"}}
                    }]).next()

print("Total number of tokens (words) uttered: {:,.0f}".format(agg['sum']))

Total number of tokens (words) uttered: 12,992,570


In [30]:
dail_4 = dail.find({"date": {"$gte": dail4["start_term"], 
                               "$lte": dail4['end_term']}}, {"text":1}).skip(1).next()

In [31]:
text = dail_4["text"]
text

["A Chinn Chomhairle agus a Theachtaí na Dála, deinim-se Liam T. Mac Cosgair d-ainmniú chun bheith 'na Uachtarán arís. Timpeall bliadhain ó shoin, thánamar, annso.  Bhí cúram na tíre orainn agus brón mór orainn mar gheall ar thrioblóid na h-aimsire sin.  Bhí orainn duine do thogha a bhéadh mar Cheann ar na Teachtaí annso agus ar an tír leis. Dheineamar fear do thogha do dhein an obair go maith.  Níor fhéadamar a rádh fé go raibh a ainm 'nár gcluasaibh le bliadhanta nó go raibh sé mar stuírtheóir agus Múinteóir duinn i bhfad roimis sin fé mar a bhí Art Ó  Gríobhtha.  Níor fhéadamar a rádh go raibh sé mar sgéul 'nár measg, dár ngriosú chun oibre, fé mar a bhí Micheál O Coileaín.  Níor fheadamar a rádh go raibh a ainm agus a chlú i n-áirde i measg muinntear na h-Éireann. Ach d'fhéadamar a rádh go raibh Liam Mac Cosgair 'na sheirbhíseach dhílis dos na daoinibh a bhí ag obair agus ag troid ar son na h-Eireann.  Dheineamar Liam Mac Cosgair do thogha mar Uachtarán.  Fear des na gnáth-daoine d

In [63]:
toks = nlp(text[3])
sentences = [[t.lemma_ for t in s if t.is_alpha and not t.is_stop] for s in toks.sents]
model = gensim.models.Word2Vec(sentences, min_count=1)

In [72]:
text[3]

'There is no person here who has standing among his own sectional group and who has responsibilities greater than the responsibilities of an ordinary member, who has not some vision of the President acting as a leader among his own people and acting as a leader among the people of Ireland, and who does not in consequence find strength in carrying out his smaller sectional duties in being a leader of a group or a party.  Personally, in the peculiar responsibility that did come to me during the term of the last Dáil, the President has been a source of help without which I did not feel that I would have been able to shoulder the responsibilities that came my way.  His help, his understanding, his insight, and the way even in which he acted as a corrective, have made very much lighter than they would have ordinarily been, responsibilities of a peculiar nature that were very great.'